<a href="https://colab.research.google.com/github/R-Sandor/crash-clock/blob/main/Abstract_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract




## Members:

Kayla Preston
Email: kpres007@odu.edu;
Web portfolio:

Raphael J. Sandor
Email: rsand006@odu.edu;
Web portfolio: https://r-sandor.github.io/

Member 3

## Raw Data Sources

### Load Data

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
from IPython.display import display, HTML

In [29]:
tempe_df=pd.read_csv('/content/drive/MyDrive/cs620-data/1.08_Crash_Data_Report_(detail)-2.csv')

In [30]:
carry_df=pd.read_json('/content/drive/MyDrive/cs620-data/cpd-crash-incidents.json')



---



### Primary Data Source
- [Tempe, AZ](https://catalog.data.gov/dataset/1-08-crash-data-report-detail-498c3):
This dataset presents the road collisions in Tempe Arrizona. This is the primary dataset that will be used to analyze daylight savings impact as the state does not observe nor adjust the UTC offset for daylight saving. The data is recorded in Arizona local time (UTC-7) which is seven hours behind UTC. The data consists of 51,304 records, with record collection starting 2012 to 2024.


#### Primary - Tempe, AZ Data Sample

In [31]:
display(HTML(tempe_df.head(2).to_html()))

,X,Y,OBJECTID,Incidentid,DateTime,Year,StreetName,CrossStreet,Distance,JunctionRelation,Totalinjuries,Totalfatalities,Injuryseverity,Collisionmanner,Lightcondition,Weather,SurfaceCondition,Unittype_One,Age_Drv1,Gender_Drv1,Traveldirection_One,Unitaction_One,Violation1_Drv1,AlcoholUse_Drv1,DrugUse_Drv1,Unittype_Two,Age_Drv2,Gender_Drv2,Traveldirection_Two,Unitaction_Two,Violation1_Drv2,AlcoholUse_Drv2,DrugUse_Drv2,Latitude,Longitude
0,-111.95221,33.42191,1,2598369.0,2012/01/01 07:25:00+00,2012.0,University Dr,Hardy Dr,0.00,Intersection Non Interchange,6.0,0.0,Non Incapacitating Injury,ANGLE (Front To Side)(Other Than Left Turn),Dark Lighted,Clear,Dry,Driver,23.0,Female,West,Making Left Turn,Inattention Distraction,Alcohol,No Apparent Influence,Driver,20.0,Female,East,Going Straight Ahead,No Improper Action,No Apparent Influence,No Apparent Influence,33.42191,-111.95221
1,-111.93769,33.42319,2,2593420.0,2012/01/01 11:45:00+00,2012.0,7th St,Myrtle Ave,139.92,Not Junction Related,0.0,0.0,No Injury,Single Vehicle,Dark Lighted,Clear,Dry,Driver,255.0,NaN,East,Backing,Unknown,No Apparent Influence,No Apparent Influence,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.42319,-111.93769


### Secondary Data Source
- [Carry, NC](https://catalog.data.gov/dataset/crash-data):
The town of Carry is located in North Carolina and participates in daylight savings time and was selected do to its approximate size to Tempe and participation Daylight Savings Time.
The raw dataset includes collisions from 2025 to 2020, consisting of over 20,500 records.

#### Carry Data Sample

In [32]:
display(HTML(carry_df.head(2).to_html()))

,tamainid,location_description,rdfeature,rdcharacter,rdclass,rdconfigur,rdsurface,rdcondition,lightcond,weather,trafcontrl,lat,lon,lon2,lat2,tract,zone,fatality,possblinj,numpassengers,numpedestrians,contrcir1_desc,contrcir2_desc,contrcir3_desc,contrcir4_desc,vehicle1,vehicle2,vehicle3,vehicle4,vehicle5,workarea,records,ta_date,ta_time,crash_date,geo_location,year,fatalities,injuries,month,contributing_factor,vehicle_type
0,44273,0 FEET FROM SR 1313 (WALNUT ST),FOUR-WAY INTERSECTION,"[STRAIGHT, LEVEL]",[STATE SECONDARY ROUTE],"[TWO-WAY, NOT DIVIDED]",SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,STOP AND GO SIGNAL,NaN,NaN,-78.7673,35.7754,None,None,0,0,2.0,NaN,NONE,None,NONE,None,PASSENGER CAR,PASSENGER CAR,None,None,None,NO,1003,2020-03-19,2025-02-01 08:38:12,2020-03-19T12:38:12+00:00,None,2020,No,No,3,"[NONE, NONE]","[PASSENGER CAR, PASSENGER CAR]"
1,44282,0 FEET FROM S.R. 1349 (DILLARD DR),FOUR-WAY INTERSECTION,"[STRAIGHT, LEVEL]",[STATE SECONDARY ROUTE],"[TWO-WAY, DIVIDED, UNPROTECTED MEDIAN]",SMOOTH ASPHALT,WET,DARK - LIGHTED ROADWAY,RAIN,STOP AND GO SIGNAL,NaN,NaN,-78.7411,35.7533,None,None,0,1,NaN,NaN,NONE,None,NONE,None,PASSENGER CAR,PASSENGER CAR,None,None,None,NO,1012,2020-03-21,2025-02-01 21:17:58,2020-03-22T01:17:58+00:00,None,2020,No,Yes,3,"[NONE, NONE]","[PASSENGER CAR, PASSENGER CAR]"


## Goal

The goal of this project is to visulize the impact of daylight savings on vehicular collisions. Recently in the United States there has been discussion at a national level adopting a permanent daylight savings time. Proponents of the legislative change claim it would decrease the number of vehicular collisions. This project sets on to analyzing and modeling the impact daylight to time of day on collisions.